In [ ]:
!pip3 install torch torchvision torchaudio mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MT_ML_Decoding/CNN/load/code') 
from load_data import load_MEG_dataset
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# X, y = load_MEG_dataset([str(i).zfill(3) for i in range(1,5)])
X_train, y_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,3)], mode = 'concatenate', training =True, output_format='numpy')
X_test, y_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,3)], mode = 'concatenate', training =False, output_format='numpy')
X_train, X_test, y_train, y_test = np.array(X_train, dtype=np.float32),\
                                   np.array(X_test, dtype=np.float32),\
                                   np.array(y_train, dtype=np.float32),\
                                   np.array(y_test, dtype=np.float32)
X_train, X_test = (X_train-X_train.mean())/X_train.std(), (X_test-X_test.mean())/X_test.std()
print('loading done.')

Loading subject 001
Download complete
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 002
Download complete
Data loaded
Subject 002 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 002
Data loaded
Subject 002 complete
--------------------------------------
loading done.


In [ ]:
ds_train = load_MEG_dataset([str(i).zfill(3) for i in range(1,3)], mode = 'concatenate', training =True, output_format='tf', batch_size=500)
ds_test = load_MEG_dataset([str(i).zfill(3) for i in range(1,3)], mode = 'concatenate', training =False, output_format='tf', batch_size=500)

Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 002
Data loaded
Subject 002 complete
--------------------------------------
Loading subject 001
Data loaded
Subject 001 complete
--------------------------------------
Loading subject 002
Data loaded
Subject 002 complete
--------------------------------------


In [ ]:
EPOCH=100
BATCH_SIZE=1
LR=1e-3
DOWNLOAD_MNIST=False  

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    # super(CNN, self)._init_()
    super(CNN, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=500,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32,64,3,1,1),
        nn.ReLU(),
        nn.MaxPool2d (2,2),
    )
    self.fc = nn.Linear(64*7*7,128)
    self.out = nn.Linear(128,10)
  def forward(self,x):
    x=self.conv1(x)
    x=self.conv2(x)
    x=x.view(x.size(0),-1)
    x=self.fc(x)
    output=self.out(x)
    return output, x

In [ ]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(17, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(cnn.parameters(), lr =LR)
loss_func=nn.CrossEntropyLoss()

In [ ]:
batch_size=50

In [ ]:
next(ds_train)

In [ ]:
for epoch in range(EPOCH):
  batch_data = next(ds_train)
  b_x = batch_data['image']
  b_y = batch_data['label']
  b_x = b_x.reshape(500,17,16,11)
  b_x = torch.tensor(b_x)
  b_y = torch.tensor(b_y)
  output = cnn(b_x)
  loss = loss_func(output, b_y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


  # batches = np.arange((X_train.shape[0]//batch_size)+1) ### Batch Indices

  # losses = [] ## Record loss of each batch
  # for batch in batches:
  #     if batch != batches[-1]:
  #         start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
  #     else:
  #         start, end = int(batch*batch_size), None

  #     X_batch, Y_batch = X_train[start:end], y_train[start:end] ## Single batch of data
  #     test_X = X_batch
  #     test_Y = Y_batch

  # if step%50 == 0:
  #   test_output, last_layer = cnn(X_test)
  #   pred_y = torch.max(test_output, 1)[1].data.numpy()
  #   accuracy=sum(pred_y==y_test)/28
  #   # accuracy = float((pred_y ==y_test.data.numpy()).astype(int).sum())/float(y_test.size(0))
  #   print('Epoch',epoch,'|train loss: %.4f' % loss.data,'|test accuracy: %.2f ' %accuracy)      
  # if(step ==1000):
  #     print("Finish 1000 images training")
  #     break
  test_output= cnn(next(ds_test)['image'])
  pred_y = torch.max(test_output, 1)[1].data.numpy()
  accuracy=sum(pred_y==next(ds_test)['label'])/28
  # accuracy = float((pred_y ==y_test.data.numpy()).astype(int).sum())/float(y_test.size(0))
  print('Epoch',epoch,'|train loss: %.4f' % loss.data,'|test accuracy: %.2f ' %accuracy)      


RuntimeError: ignored

In [ ]:

for i in range(1, EPOCH+1):
    batches = np.arange((X_train.shape[0]//batch_size)+1) ### Batch Indices

    losses = [] ## Record loss of each batch
    for batch in batches:
        if batch != batches[-1]:
            start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
        else:
            start, end = int(batch*batch_size), None

        X_batch, Y_batch = X_train[start:end], y_train[start:end] ## Single batch of data
        test_X = X_batch
        test_Y = Y_batch
        output = cnn(X_batch)[0]
        loss = loss_func(output, Y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #params = jax.tree_map(UpdateWeights, params, param_grads) ## Update Params
        losses.append(loss) ## Record Loss


    # summary_writer.scalar('train_loss', jnp.array(losses).mean(), i)
    print("CrossEntropy Loss : {:.2f}".format(np.array(losses).mean()))

TypeError: ignored

In [ ]:
test_output, _ = cnn(X_test[:10])
pred_y =torch.max(test_output, 1)[1].data.numpy()
print(pred_y, 'prediction number')
print(y_test[:10].numpy(),'real number')